In [13]:
import numpy as np
import matplotlib.pyplot as plt
import dask
from dask import delayed, compute
from Bio import SeqIO
import time
import json

# File paths
fastq_file = "16S_WT_day3_11_SRR2628505_1.fastq"
kmer_freq_file = "kmer_frequencies.txt"
de_bruijn_file = "de_bruijn_graph.txt"
k = 3  # K-mer size

# Step 1: Extract K-mers and Save Frequencies
def extract_kmers(sequence, k):
    """
    Extract k-mers from a given sequence and return their frequencies along with predecessors and successors.
    """
    kmers = {}
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i + k]
        predecessor = sequence[i - 1] if i > 0 else None
        successor = sequence[i + k] if i + k < len(sequence) else None
        
        if kmer not in kmers:
            kmers[kmer] = {'count': 0, 'predecessors': set(), 'successors': set()}
        
        kmers[kmer]['count'] += 1
        if predecessor:
            kmers[kmer]['predecessors'].add(predecessor)
        if successor:
            kmers[kmer]['successors'].add(successor)
    
    return kmers

def save_kmer_frequencies(kmer_frequencies, filepath):
    """
    Save k-mer frequencies to a text file in JSON format for future retrieval.
    Converts sets (predecessors, successors) into lists for JSON compatibility.
    """
    # Convert sets to lists for JSON serialization
    serializable_kmers = {
        kmer: {
            'count': data['count'],
            'predecessors': list(data['predecessors']),
            'successors': list(data['successors'])
        }
        for kmer, data in kmer_frequencies.items()
    }

    with open(filepath, 'w') as file:
        json.dump(serializable_kmers, file)


def load_kmer_frequencies(filepath):
    """
    Load k-mer frequencies from a text file and convert lists back to sets.
    """
    with open(filepath, 'r') as file:
        loaded_kmers = json.load(file)

    # Convert lists back to sets for predecessors and successors
    kmer_frequencies = {
        kmer: {
            'count': data['count'],
            'predecessors': set(data['predecessors']),
            'successors': set(data['successors'])
        }
        for kmer, data in loaded_kmers.items()
    }

    return kmer_frequencies


def parse_fastq_and_save_kmers(fastq_file, k):
    """
    Parse FASTQ file, extract k-mers, and save the frequencies to a text file.
    """
    kmer_frequencies = {}
    
    with open(fastq_file, "r") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            sequence = str(record.seq)
            kmers = extract_kmers(sequence, k)
            
            for kmer, data in kmers.items():
                if kmer not in kmer_frequencies:
                    kmer_frequencies[kmer] = {'count': 0, 'predecessors': set(), 'successors': set()}
                kmer_frequencies[kmer]['count'] += data['count']
                kmer_frequencies[kmer]['predecessors'].update(data['predecessors'])
                kmer_frequencies[kmer]['successors'].update(data['successors'])
    
    save_kmer_frequencies(kmer_frequencies, kmer_freq_file)
    return kmer_frequencies

# Step 2: Pearson Skewness Calculation
def calculate_pearson_skewness(frequencies):
    """
    Calculate the Pearson skewness coefficient based on the frequencies of k-mers.
    """
    mean = np.mean(frequencies)
    median = np.median(frequencies)
    std_dev = np.std(frequencies)
    skewness = 3 * (mean - median) / std_dev if std_dev != 0 else 0
    return skewness

# Step 3: Save/Load De Bruijn Graph
def build_de_bruijn_graph(kmer_frequencies):
    """
    Build the De Bruijn graph using k-mer frequencies, where each k-mer points to its successors.
    """
    de_bruijn_graph = {}
    for kmer, data in kmer_frequencies.items():
        if len(data['successors']) == 1:
            de_bruijn_graph[kmer] = next(iter(data['successors']))
    return de_bruijn_graph

def save_de_bruijn_graph(de_bruijn_graph, filepath):
    """
    Save De Bruijn graph to a text file in JSON format.
    """
    with open(filepath, 'w') as file:
        json.dump(de_bruijn_graph, file)

def load_de_bruijn_graph(filepath):
    """
    Load De Bruijn graph from a text file.
    """
    with open(filepath, 'r') as file:
        return json.load(file)

# Step 4: Plot k-mer Frequency Histogram
def plot_kmer_histogram(kmer_frequencies, read_index):
    """
    Plot the k-mer frequency distribution as a histogram.
    """
    plt.figure(figsize=(10, 6))
    kmer_list = list(kmer_frequencies.keys())
    frequency_list = [data['count'] for data in kmer_frequencies.values()]
    
    plt.bar(kmer_list, frequency_list)
    plt.xlabel('k-mer')
    plt.ylabel('Frequency')
    plt.title(f'k-mer Frequency in Read {read_index}')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

# Step 5: Parallel Genome Assembly with Dask
@delayed
def traverse_kmers(adjacency_list, starting_kmer):
    """
    Traverse from a starting k-mer to form a segment of the genome.
    """
    genome_segment = starting_kmer
    current_kmer = starting_kmer
    
    while current_kmer in adjacency_list:
        next_kmer = adjacency_list[current_kmer]
        genome_segment += next_kmer[-1]
        current_kmer = next_kmer
    
    return genome_segment

def traverse_de_bruijn_graph(de_bruijn_graph, starting_kmer):
    """
    Traverse the de Bruijn graph from a starting k-mer to form a segment of the genome.
    """
    genome_segment = starting_kmer
    current_kmer = starting_kmer
    
    while current_kmer in de_bruijn_graph:
        # Get the next k-mer (successor)
        successors = de_bruijn_graph[current_kmer]
        if len(successors) != 1:
            # If there's ambiguity, stop the traversal
            break
        next_kmer = next(iter(successors))
        genome_segment += next_kmer[-1]  # Append only the last character
        current_kmer = next_kmer
    
    return genome_segment

@delayed
def parallel_traverse_kmers(de_bruijn_graph, starting_kmer):
    """
    Wrapper for the traverse function that allows Dask to parallelize it.
    """
    return traverse_de_bruijn_graph(de_bruijn_graph, starting_kmer)

def parallel_genome_construction(de_bruijn_graph):
    """
    Reconstruct the genome by parallelizing k-mer traversal using Dask.
    """
    tasks = []
    
    # Step 1: Traverse the de Bruijn graph from each k-mer in parallel
    for starting_kmer in de_bruijn_graph.keys():
        tasks.append(parallel_traverse_kmers(de_bruijn_graph, starting_kmer))
    
    # Step 2: Compute the tasks to get the genome segments
    contig = compute(*tasks)
    print(contig)

# Main Script
def main():
    start_time = time.time()
    
    # Step 1: Parse FASTQ and Save K-mer Frequencies
    print("Extracting and saving k-mers...")
    kmer_frequencies = parse_fastq_and_save_kmers(fastq_file, k)
    
    # Step 2: Calculate Skewness and Identify Significant Skew
    # skew_threshold = 1  # Skewness threshold for identifying significant skew
    # for read_index, record in enumerate(SeqIO.parse(fastq_file, "fastq")):
    #     sequence = str(record.seq)
    #     kmer_frequencies = extract_kmers(sequence, k)
    #     frequencies = [data['count'] for data in kmer_frequencies.values()]
    #     skewness = calculate_pearson_skewness(frequencies)
    #     print(f"Read {read_index + 1}: Skewness = {skewness}")
        
    #     if abs(skewness) > skew_threshold:
    #         print(f"Read {read_index + 1} has significant skewness.")
    #         plot_kmer_histogram(kmer_frequencies, read_index + 1)
    
    # Step 3: Build and Save De Bruijn Graph
    print("Building and saving De Bruijn graph...")
    de_bruijn_graph = build_de_bruijn_graph(kmer_frequencies)
    save_de_bruijn_graph(de_bruijn_graph, de_bruijn_file)
    
    # Step 4: Parallel Genome Construction
    print("Reconstructing genome in parallel...")
    parallel_genome_construction(de_bruijn_graph)
    
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Extracting and saving k-mers...
Building and saving De Bruijn graph...
Reconstructing genome in parallel...
('GNAG', 'GCNG', 'AGNG', 'NGTT', 'CNAG', 'ACNN', 'CNNN', 'NNCC', 'NCCN', 'NNGG', 'GTNA', 'TNAT', 'ATNC', 'TNCG', 'NCGT', 'NANG', 'GANN', 'ANNN', 'NAAA', 'CANG')
Elapsed time: 3.83 seconds
